In [1]:
import tensorflow as tf
import csv
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
from tensorflow import keras

In [2]:
def label_to_int(label):
    labels = {'A' : 0, 'B' : 1, 'C' : 2, 'D' : 3, 'E' : 4, 'F' : 5, 'G' : 6, 'H' : 7, 'I' : 8, 'J' : 9}
    return labels[label]

In [3]:
import random
def split(images, labels):
    test_indexes = random.sample(range(len(images)), len(images) // 10)
    train_indexes = set([i for i in range(len(images))]) - set(test_indexes)
    return [images[i] for i in train_indexes], [labels[i] for i in train_indexes], [images[i] for i in test_indexes], [labels[i] for i in test_indexes]

In [4]:
def read_points():
    images = []
    labels = []
    for path, subdirs, files in os.walk("../Datasets/notMNIST_small"):
        for name in files:
            try:
                Image.open(os.path.join(path, name), 'r')
                images.append(os.path.join(path, name))
                labels.append(label_to_int(path.split("/")[-1]))
            except:
                pass
        
    train_images, train_labels, test_images, test_labels = split(images, labels)
    train_points = []
    new_train_labels = []
    test_points = []
    new_test_labels = []
    
    for j in range(len(train_images)):
        try:
            im = Image.open(train_images[j], 'r')
            im1 = im.rotate(10)
            im2 = im.rotate(-10)
            width, height = im.size
            im3 = im.resize((width - 10, height - 10)).resize((width, height))
            pixel_values = list(im.getdata())
            pixel_values1 = list(im1.getdata())
            pixel_values2 = list(im2.getdata())
            pixel_values3 = list(im3.getdata())
            for i in range(len(pixel_values)):
                pixel_values[i] = pixel_values[i] * 1.0 / 255
            for i in range(len(pixel_values1)):
                pixel_values1[i] = pixel_values1[i] * 1.0 / 255
            for i in range(len(pixel_values2)):
                pixel_values2[i] = pixel_values2[i] * 1.0 / 255
            for i in range(len(pixel_values3)):
                pixel_values3[i] = pixel_values3[i] * 1.0 / 255
            train_points.append(np.reshape(pixel_values, [width, height]))
            train_points.append(np.reshape(pixel_values1, [width, height]))
            train_points.append(np.reshape(pixel_values2, [width, height]))
            train_points.append(np.reshape(pixel_values3, [width, height]))
            new_train_labels.append(train_labels[j])
            new_train_labels.append(train_labels[j])
            new_train_labels.append(train_labels[j])
            new_train_labels.append(train_labels[j])
        except:
            pass 

    train_points, train_labels = np.array(train_points), np.array(new_train_labels)
    permutation = np.arange(train_points.shape[0])
    np.random.shuffle(permutation)
    train_points, train_labels = train_points[permutation], train_labels[permutation]
    
    for j in range(len(test_images)):
        try:
            im = Image.open(test_images[j], 'r')
            width, height = im.size
            pixel_values = list(im.getdata())
            for i in range(len(pixel_values)):
                pixel_values[i] = pixel_values[i] * 1.0 / 255
            test_points.append(np.reshape(pixel_values, [width, height]))
            new_test_labels.append(test_labels[j])
        except:
            pass
    
    test_points, test_labels = np.array(test_points), np.array(new_test_labels)
    return train_points, train_labels, test_points, test_labels

In [5]:
train_points, train_labels, test_points, test_labels = read_points()

In [6]:
print(len(train_points) * 1.0 / 32)

2106.5


In [7]:
def model(features, labels, mode):
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])
    conv1 = keras.layers.Conv2D(8, [3, 3], padding="same", activation=tf.nn.relu)(input_layer)
    conv2 = keras.layers.Conv2D(8, [3, 3], padding="same", activation=tf.nn.relu)(conv1)
    conv3 = keras.layers.Conv2D(8, [3, 3], padding="same", activation=tf.nn.relu)(conv2)
    conv4 = keras.layers.Conv2D(8, [3, 3], padding="same", activation=tf.nn.relu)(conv3)
    
    conv4_reshaped = tf.reshape(conv4, [-1, 28 * 28 * 8])
    
    fc1 = keras.layers.Dense(64, activation=tf.tanh)(conv4_reshaped)
    fc2 = keras.layers.Dense(64, activation=tf.tanh)(fc1)
    logits = keras.layers.Dense(10)(fc2)
    
    predictions = {
        "classes": tf.argmax(input=logits, axis=1)
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
        train_op = optimizer.minimize(loss=loss,global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])
    }
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [8]:
def train_and_validate(train_points, train_labels, validate_points, validate_labels, dir_name, N):
    mnist_classifier = tf.estimator.Estimator(
        model_fn=model, model_dir=dir_name)
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": train_points},
        y=train_labels,
        batch_size=32,
        num_epochs=None,
        shuffle=True)

    for epoch in range(N):
        mnist_classifier.train(input_fn=train_input_fn, steps=1872)
        eval_input_fn_train = tf.estimator.inputs.numpy_input_fn(
            x={"x": train_points},
            y=train_labels,
            num_epochs=1,
            shuffle=False)
        eval_results_train = mnist_classifier.evaluate(input_fn=eval_input_fn_train)
        
        eval_input_fn_test = tf.estimator.inputs.numpy_input_fn(
            x={"x": test_points},
            y=test_labels,
            num_epochs=1,
            shuffle=False)
        eval_results_test = mnist_classifier.evaluate(input_fn=eval_input_fn_test)
        print("Epoch " + str(epoch) + ": train accuracy = " + str(eval_results_train["accuracy"]) + ", test accuracy = " + str(eval_results_test["accuracy"]))
        if eval_results_test["accuracy"] >= 0.95:
            break

In [9]:
tf.logging.set_verbosity(tf.logging.ERROR)

In [ ]:
train_and_validate(train_points, train_labels, test_points, test_labels,"not_mnist_model", 40)

Epoch 0: train accuracy = 0.93454784, test accuracy = 0.9230769
Epoch 1: train accuracy = 0.94610435, test accuracy = 0.9257479
